In [6]:
#Creacion de las rutas 

import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

DATA_PATH = r'C:\Users\Administrador\Desktop\DATos\Ciclo 1\Recursos DN_COM_58'

FILE_CUSTOMERS = 'olist_customers_dataset.xlsx'
FILE_GEOLOCATIONS = 'olist_geolocation_dataset.csv'

### completa los nombres del resto de los archivos con su extesion (ejemplo .csv) ...
FILE_ITEMS =  'olist_order_items_dataset.csv'
FILE_PAYMENTS = 'olist_order_payments_dataset.csv'
FILE_ORDERS = 'olist_orders_dataset.csv'
FILE_STATES_ABBREVIATIONS = 'states_abbreviations.json'


In [7]:
geolocations = pd.read_csv(
    os.path.join(DATA_PATH, FILE_GEOLOCATIONS),
    dtype={'geolocation_zip_code_prefix': 'str'}
    )

In [12]:
%pip install openpyxl
customers = pd.read_excel(
    os.path.join(DATA_PATH, FILE_CUSTOMERS),
    engine='openpyxl',
    # Especificar el tipo de dato de customer_zip_code_prefix
    dtype={'customer_zip_code_prefix':'str'} 
    )

items = pd.read_csv(
    os.path.join(DATA_PATH, FILE_ITEMS)
    )

items_agg = items.groupby(
    ['order_id']).agg(
        # conteo de producto
        {'order_item_id': 'count',
        # suma de los precios de los artículos
        'price': 'sum'}
                      ).reset_index() 
items_agg.rename(
    columns={'order_item_id': 'total_products', 'price': 'total_sales'},
    inplace=True
    )

payments = pd.read_csv(
    os.path.join(DATA_PATH, FILE_PAYMENTS)
    )

states_abbreviations = pd.read_json(
     os.path.join(DATA_PATH, FILE_STATES_ABBREVIATIONS))

orders = pd.read_csv(
     os.path.join(DATA_PATH, FILE_ORDERS))
## convierte order_purchase_timestamp
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'], errors='coerce' )

# order_approved_at
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'], errors='coerce' )

# order_delivered_carrier_date
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'], errors='coerce' )

# order_delivered_customer_date
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'], errors='coerce' )

# order_estimated_delivery_date
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'], errors='coerce' )


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [13]:
orders['delta_days'] = (
    orders['order_delivered_customer_date'] -
    orders['order_estimated_delivery_date']
    ).dt.total_seconds()/ 60 / 60 / 24

orders['delay_status']  = np.where(
    orders['delta_days'] > 3, 'long_delay',
    np.where(orders['delta_days'] <= 0, 'on_time','short_delay')
    )

In [15]:
geolocations.query("geolocation_zip_code_prefix == 24220")
unique_geolocations = geolocations.drop_duplicates(
    subset = ['geolocation_zip_code_prefix']
    )
unique_geolocations.query(
    "geolocation_zip_code_prefix == 24220"
    )

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state


In [20]:
customers_geolocation = customers.merge(
    unique_geolocations,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)
customers_geolocation.columns

customers_geolocation_estado = customers_geolocation.merge(
    states_abbreviations,
    left_on='geolocation_state',
    right_on='abbreviation',
    how='left'
)
orders_totals = orders.merge(
    items_agg,
    on=['order_id'],
    how='left'
    )

results = orders_totals.merge(
    customers_geolocation_estado,
    on=['customer_id'],
    how='left'
    )
results.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'distance_distribution_center', 'delta_days', 'delay_status',
       'total_products', 'total_sales', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'geolocation_city', 'geolocation_state', 'abbreviation', 'state_name'],
      dtype='object')

In [ ]:
# Completa el codigo
results.to_csv(
    # nombre del archivo
    'oilst_processed.csv',
    # flag para no escribir el indice del dataframe al csv
    index=False
    )